In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
from xgboost import XGBClassifier
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.metrics import accuracy_score
import joblib
import os

In [2]:
pd.set_option('display.max_columns', None)
df = pd.read_csv("Blood_samples_dataset_balanced_2(f).csv")
df.head()

,Glucose,Cholesterol,Hemoglobin,Platelets,White Blood Cells,Red Blood Cells,Hematocrit,Mean Corpuscular Volume,Mean Corpuscular Hemoglobin,Mean Corpuscular Hemoglobin Concentration,Insulin,BMI,Systolic Blood Pressure,Diastolic Blood Pressure,Triglycerides,HbA1c,LDL Cholesterol,HDL Cholesterol,ALT,AST,Heart Rate,Creatinine,Troponin,C-reactive Protein,Disease
0,0.739597,0.650198,0.713631,0.868491,0.687433,0.529895,0.290006,0.631045,0.001328,0.795829,0.034129,0.071774,0.185596,0.071455,0.653472,0.502665,0.215560,0.512941,0.064187,0.610827,0.939485,0.095512,0.465957,0.769230,Healthy
1,0.121786,0.023058,0.944893,0.905372,0.507711,0.403033,0.164216,0.307553,0.207938,0.505562,0.571162,0.839271,0.580903,0.556037,0.477742,0.856810,0.652465,0.106961,0.942549,0.344261,0.666368,0.659060,0.816982,0.401166,Diabetes
2,0.452539,0.116135,0.544560,0.400640,0.294538,0.382021,0.625267,0.295122,0.868369,0.026808,0.582170,0.577423,0.045782,0.663642,0.088940,0.466795,0.387332,0.421763,0.007186,0.506918,0.431704,0.417295,0.799074,0.779208,Thalasse
3,0.136609,0.015605,0.419957,0.191487,0.081168,0.166214,0.073293,0.668719,0.125447,0.501051,0.620258,0.169059,0.139289,0.160496,0.406528,0.016256,0.040137,0.826721,0.265415,0.594148,0.225756,0.490349,0.637061,0.354094,Anemia
4,0.176737,0.752220,0.971779,0.785286,0.443880,0.439851,0.894991,0.442159,0.257288,0.805987,0.184371,0.580175,0.118461,0.005579,0.414407,0.429431,0.146294,0.221574,0.015280,0.567115,0.841412,0.153350,0.794008,0.094970,Thalasse


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2351 entries, 0 to 2350
Data columns (total 25 columns):
 #   Column                                     Non-Null Count  Dtype  
---  ------                                     --------------  -----  
 0   Glucose                                    2351 non-null   float64
 1   Cholesterol                                2351 non-null   float64
 2   Hemoglobin                                 2351 non-null   float64
 3   Platelets                                  2351 non-null   float64
 4   White Blood Cells                          2351 non-null   float64
 5   Red Blood Cells                            2351 non-null   float64
 6   Hematocrit                                 2351 non-null   float64
 7   Mean Corpuscular Volume                    2351 non-null   float64
 8   Mean Corpuscular Hemoglobin                2351 non-null   float64
 9   Mean Corpuscular Hemoglobin Concentration  2351 non-null   float64
 10  Insulin                 

In [4]:
df["Disease"].value_counts()

Disease
Anemia      623
Healthy     556
Diabetes    540
Thalasse    509
Thromboc    123
Name: count, dtype: int64

In [5]:
df[df["Disease"]=="Thromboc"].nunique()

Glucose                                      3
Cholesterol                                  3
Hemoglobin                                   3
Platelets                                    3
White Blood Cells                            3
Red Blood Cells                              3
Hematocrit                                   3
Mean Corpuscular Volume                      3
Mean Corpuscular Hemoglobin                  3
Mean Corpuscular Hemoglobin Concentration    3
Insulin                                      3
BMI                                          3
Systolic Blood Pressure                      3
Diastolic Blood Pressure                     3
Triglycerides                                3
HbA1c                                        3
LDL Cholesterol                              3
HDL Cholesterol                              3
ALT                                          3
AST                                          3
Heart Rate                                   3
Creatinine   

In [6]:
df[df["Disease"]=="Thromboc"].describe()

,Glucose,Cholesterol,Hemoglobin,Platelets,White Blood Cells,Red Blood Cells,Hematocrit,Mean Corpuscular Volume,Mean Corpuscular Hemoglobin,Mean Corpuscular Hemoglobin Concentration,Insulin,BMI,Systolic Blood Pressure,Diastolic Blood Pressure,Triglycerides,HbA1c,LDL Cholesterol,HDL Cholesterol,ALT,AST,Heart Rate,Creatinine,Troponin,C-reactive Protein
count,123.000000,123.000000,123.000000,123.000000,123.000000,123.000000,123.000000,123.000000,123.000000,123.000000,123.000000,123.000000,123.000000,123.000000,123.000000,123.000000,123.000000,123.000000,123.000000,123.000000,123.000000,123.000000,123.000000,123.000000
mean,0.275675,0.634323,0.774260,0.095876,0.594539,0.632704,0.601235,0.416061,0.457412,0.670721,0.365144,0.273549,0.298547,0.395217,0.516482,0.523185,0.313274,0.747346,0.581704,0.419925,0.759035,0.559980,0.520617,0.585555
std,0.129755,0.101355,0.015666,0.071395,0.291473,0.172480,0.165284,0.035165,0.233908,0.109790,0.222993,0.123802,0.257676,0.141188,0.163887,0.093025,0.318535,0.106490,0.214669,0.199472,0.168923,0.144763,0.119301,0.041066
min,0.107165,0.508254,0.755007,0.012594,0.227684,0.425117,0.387461,0.380656,0.248189,0.490245,0.151013,0.119338,0.142591,0.212859,0.391481,0.446854,0.033037,0.615543,0.253892,0.233890,0.612188,0.407891,0.426863,0.532100
25%,0.107165,0.508254,0.755007,0.012594,0.227684,0.425117,0.387461,0.380656,0.248189,0.490245,0.151013,0.119338,0.142591,0.212859,0.391481,0.446854,0.033037,0.615543,0.253892,0.233890,0.612188,0.407891,0.426863,0.532100
50%,0.353734,0.603341,0.777394,0.096310,0.718674,0.652505,0.684896,0.396669,0.305588,0.732671,0.232877,0.306056,0.143713,0.474378,0.393263,0.489514,0.102679,0.772045,0.609068,0.322486,0.659069,0.554960,0.472428,0.609177
75%,0.399017,0.757757,0.791215,0.178840,0.925520,0.825769,0.753657,0.461418,0.762667,0.741120,0.654441,0.403920,0.722350,0.535940,0.731369,0.673188,0.729376,0.861035,0.794735,0.677415,0.979192,0.774219,0.714177,0.621687
max,0.399017,0.757757,0.791215,0.178840,0.925520,0.825769,0.753657,0.461418,0.762667,0.741120,0.654441,0.403920,0.722350,0.535940,0.731369,0.673188,0.729376,0.861035,0.794735,0.677415,0.979192,0.774219,0.714177,0.621687


In [7]:
df[df["Disease"]=="Thalasse"].describe()

,Glucose,Cholesterol,Hemoglobin,Platelets,White Blood Cells,Red Blood Cells,Hematocrit,Mean Corpuscular Volume,Mean Corpuscular Hemoglobin,Mean Corpuscular Hemoglobin Concentration,Insulin,BMI,Systolic Blood Pressure,Diastolic Blood Pressure,Triglycerides,HbA1c,LDL Cholesterol,HDL Cholesterol,ALT,AST,Heart Rate,Creatinine,Troponin,C-reactive Protein
count,509.000000,509.000000,509.000000,509.000000,509.000000,509.000000,509.000000,509.000000,509.000000,509.000000,509.000000,509.000000,509.000000,509.000000,509.000000,509.000000,509.000000,509.000000,509.000000,509.000000,509.000000,509.000000,509.000000,509.000000
mean,0.361553,0.334211,0.636288,0.474129,0.605361,0.548459,0.549945,0.523751,0.690439,0.499853,0.420885,0.511786,0.352612,0.353903,0.275963,0.421222,0.401945,0.512812,0.510697,0.449088,0.564475,0.380793,0.395739,0.441164
std,0.216243,0.201458,0.268687,0.269478,0.218802,0.207174,0.235326,0.295769,0.317037,0.342773,0.205727,0.239933,0.242077,0.279671,0.200009,0.268183,0.202542,0.284364,0.275933,0.215308,0.210395,0.215466,0.261552,0.286022
min,0.016514,0.089600,0.027259,0.068281,0.294538,0.263589,0.232516,0.103437,0.025514,0.006947,0.051976,0.063970,0.005988,0.005579,0.012693,0.022621,0.061317,0.200323,0.007186,0.013013,0.298837,0.076333,0.012287,0.030796
25%,0.185553,0.124738,0.487907,0.189737,0.402035,0.382021,0.329199,0.292981,0.257288,0.218293,0.251957,0.358225,0.162423,0.111589,0.088940,0.151173,0.261072,0.221574,0.363050,0.364597,0.339125,0.153350,0.159601,0.094970
50%,0.407101,0.300282,0.609836,0.476448,0.566526,0.479954,0.494564,0.442159,0.866320,0.460602,0.425386,0.577423,0.392288,0.216218,0.299213,0.466795,0.387332,0.421763,0.621086,0.506918,0.544136,0.383653,0.382066,0.572092
75%,0.531424,0.442105,0.851987,0.754841,0.837087,0.729285,0.851338,0.820546,0.915391,0.805987,0.582170,0.749109,0.546705,0.674623,0.414407,0.708808,0.518181,0.819674,0.727706,0.570902,0.800666,0.494329,0.566289,0.633088
max,0.731786,0.752220,0.983306,0.991188,0.990786,0.992377,0.913721,0.995263,0.921178,0.975586,0.797331,0.789497,0.747593,0.761455,0.630936,0.798970,0.798043,0.952549,0.790341,0.767707,0.871053,0.798090,0.799074,0.797906


In [8]:


target = "Disease"   # <-- change to your label column

features = df.columns.drop(target)

rules = {}

for col in features:
    col_rules = {}
    for disease in df[target].unique():
        subset = df[df[target] == disease]

        # ignore categorical columns
        if not np.issubdtype(df[col].dtype, np.number):
            continue

        # find min/max for that disease
        min_val = subset[col].min()
        max_val = subset[col].max()

        col_rules[disease] = (min_val, max_val)

    rules[col] = col_rules

rules


{'Glucose': {'Healthy': (np.float64(0.0425626291856029),
   np.float64(0.7992818548336882)),
  'Diabetes': (np.float64(0.0129560088675246), np.float64(0.9684602088259044)),
  'Thalasse': (np.float64(0.0165135568266396), np.float64(0.7317857384038666)),
  'Anemia': (np.float64(0.0109939277897717), np.float64(0.7891114760850271)),
  'Thromboc': (np.float64(0.1071651041947874),
   np.float64(0.3990167583554709))},
 'Cholesterol': {'Healthy': (np.float64(0.0552898586358352),
   np.float64(0.7306858460671382)),
  'Diabetes': (np.float64(0.0121393863118908), np.float64(0.9050264233180664)),
  'Thalasse': (np.float64(0.0896001919198326), np.float64(0.7522199398931788)),
  'Anemia': (np.float64(0.0156053585175475), np.float64(0.7839180287374858)),
  'Thromboc': (np.float64(0.5082540359498127),
   np.float64(0.7577568401935213))},
 'Hemoglobin': {'Healthy': (np.float64(0.1904851029640255),
   np.float64(0.964962790337339)),
  'Diabetes': (np.float64(0.0776179839504562), np.float64(0.98223053820

In [9]:
df.nunique()


Glucose                                      65
Cholesterol                                  65
Hemoglobin                                   65
Platelets                                    65
White Blood Cells                            65
Red Blood Cells                              65
Hematocrit                                   65
Mean Corpuscular Volume                      65
Mean Corpuscular Hemoglobin                  65
Mean Corpuscular Hemoglobin Concentration    65
Insulin                                      65
BMI                                          65
Systolic Blood Pressure                      65
Diastolic Blood Pressure                     65
Triglycerides                                65
HbA1c                                        65
LDL Cholesterol                              65
HDL Cholesterol                              65
ALT                                          65
AST                                          65
Heart Rate                              

In [10]:
test_df = pd.read_csv("Blood_samples_dataset_test.csv")
df_concat = pd.concat([df,test_df], axis=0)
mapping = {"Anemia":0, "Healthy":1, "Diabetes":2,"Thalasse":3,"Thromboc":4,"Heart Di":5}
df_concat['Disease'] = df_concat['Disease'].map(mapping)

In [11]:
from sklearn.model_selection import train_test_split


train_col = [col for col in df_concat.columns if col != 'Disease']
X = df_concat[train_col]  
y = df_concat['Disease']  


X_train, X_test, y_train, y_test = train_test_split(
    X, 
    y, 
    test_size=0.2,      
    stratify=y,         
    random_state=42
)


In [12]:
from sklearn.model_selection import StratifiedKFold
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
import numpy as np

# Features and target
X = df_concat.drop('Disease', axis=1)
y = df_concat['Disease']

# Stratified 5-fold cross-validation
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
fold_accuracies = []

for fold, (train_index, test_index) in enumerate(skf.split(X, y), 1):
    # Split the data for this fold
    X_train_fold, X_test_fold = X.iloc[train_index], X.iloc[test_index]
    y_train_fold, y_test_fold = y.iloc[train_index], y.iloc[test_index]
    
    # Initialize a new XGBoost model for this fold
    model = XGBClassifier(
        eval_metric='mlogloss',
        max_depth=5,
        learning_rate=0.3,
        n_estimators=121,
        subsample=0.88,
        colsample_bytree=0.8,
        random_state=42
    )
    
    # Train on the training fold
    model.fit(X_train_fold, y_train_fold)
    
    # Predict on the test fold
    y_pred_fold = model.predict(X_test_fold)
    
    # Compute accuracy for this fold
    acc = accuracy_score(y_test_fold, y_pred_fold)
    fold_accuracies.append(acc)
    print(f"Fold {fold} Accuracy: {acc:.4f}")

# Mean accuracy across all folds
print("Mean CV Accuracy:", np.mean(fold_accuracies))


Fold 1 Accuracy: 0.9771
Fold 2 Accuracy: 0.9859
Fold 3 Accuracy: 0.9877
Fold 4 Accuracy: 0.9841
Fold 5 Accuracy: 0.9841
Mean CV Accuracy: 0.9837872916511413


In [13]:
from sklearn.metrics import accuracy_score
from xgboost import XGBClassifier

model = XGBClassifier(
    eval_metric='mlogloss',
    max_depth=5,
    learning_rate=0.3,
    n_estimators=121,
    subsample=0.88,
    colsample_bytree=0.8,
    random_state=42
)

model.fit(X_train, y_train)
y_pred = model.predict(X_test)

acc = accuracy_score(y_test, y_pred)
print("Accuracy:", acc)


Accuracy: 0.9859154929577465


In [14]:
# Train final XGBoost model on all data and save as .pkl for backend

# Reuse the concatenated dataset and mapping created above
if "df_concat" not in globals():
    raise ValueError("df_concat not found. Please run the preprocessing cells first.")

X_full = df_concat.drop("Disease", axis=1)
y_full = df_concat["Disease"]

# Train final model on the full dataset
final_model = XGBClassifier(
    eval_metric="mlogloss",
    max_depth=5,
    learning_rate=0.3,
    n_estimators=121,
    subsample=0.88,
    colsample_bytree=0.8,
    random_state=42,
)

final_model.fit(X_full, y_full)

# Prepare bundle to save: model + feature names + label mapping
feature_cols = X_full.columns.tolist()

# `mapping` was defined earlier when encoding the Disease column
if "mapping" not in globals():
    raise ValueError("label mapping dict 'mapping' not found. Run the encoding cell first.")

bundle = {
    "model": final_model,
    "feature_cols": feature_cols,
    "label_mapping": mapping,
}

# Ensure models directory exists
models_dir = "models"
os.makedirs(models_dir, exist_ok=True)

model_path = os.path.join(models_dir, "mediguard_model.pkl")
joblib.dump(bundle, model_path)

print(f"Saved MediGuard model bundle to: {model_path}")

Saved MediGuard model bundle to: models/mediguard_model.pkl
